In [110]:
from datetime import datetime
import pytz


def get_current_date():

    # Define CET timezone
    cet_timezone = pytz.timezone("Europe/Berlin")

    # Get current time in CET
    current_cet_date = datetime.now(cet_timezone).date()

    return current_cet_date


current_date = get_current_date()

In [111]:
link = "https://www.kusss.jku.at/kusss/published-calendar.action?token=yfqJ0X1_NO87gomi3JMOBuMNPaDIMOSEZNoQVfYt&lang=en"

In [113]:
import requests
from ics import Calendar


def get_today_events(current_date):
    response = requests.get(link)
    calendar = Calendar(response.text)

    today_events = []

    for event in calendar.events:
        calendar_date = event.begin.date()
        if calendar_date==current_date:
            today_events.append(event)

    # Sorting events by time, from earliest
    today_events = sorted(today_events, key=lambda event: event.begin)
    
    return today_events

today_events = get_today_events(current_date)

In [114]:
def get_daily_schedule(date, events):
    # Template for the header
    header_template = (
        "📅 Good Morning, Students!\n"
        "Here’s your schedule for {date}:\n\n"
    )
    
    # Template for each event (subject/class)
    event_template = (
        "━━━━━━━━━━━━━━━━━━━━━━━\n"
        "🎯 Subject: {course_title}\n"
        "👨‍🏫 Instructor: {instructor}\n"
        "🗓 Time: {start_time} – {end_time}\n"
        "🏫 Room: {location}\n"
    )
    
    # Start the message with the header
    message = header_template.format(date=date)
    
    # Add each event to the message
    for event in events:
        # Extract details from ICS event
        summary = event.name  # e.g. "KV Responsible AI / Martina Mara / (510101/2024W)"
        parts = summary.split(" / ")
        
        if len(parts) >= 2:
            course_title = parts[0].strip()
            instructor = parts[1].strip()
        else:
            # If the format is different or incomplete
            course_title = summary.strip()
            instructor = "N/A"
        
        start_time = event.begin.format('HH:mm')
        end_time = event.end.format('HH:mm')
        location = event.location if event.location else "N/A"
        
        message += event_template.format(
            course_title=course_title,
            instructor=instructor,
            start_time=start_time,
            end_time=end_time,
            location=location
        )
        
    
    return message

message = get_daily_schedule(current_date, today_events)
print(message)


📅 Good Morning, Students!
Here’s your schedule for 2024-12-16:

━━━━━━━━━━━━━━━━━━━━━━━
🎯 Subject: VL Hands-on AI I
👨‍🏫 Instructor: Rainer Dangl
🗓 Time: 08:30 – 10:00
🏫 Room: HS 7    
━━━━━━━━━━━━━━━━━━━━━━━
🎯 Subject: UE Hands-on AI I
👨‍🏫 Instructor: Sohvi Iiri Maria Luukkonen
🗓 Time: 10:15 – 11:45
🏫 Room: S5 103  
━━━━━━━━━━━━━━━━━━━━━━━
🎯 Subject: KV Responsible AI
👨‍🏫 Instructor: Martina Mara
🗓 Time: 12:00 – 13:30
🏫 Room: HS 7    
━━━━━━━━━━━━━━━━━━━━━━━
🎯 Subject: UE Logic
👨‍🏫 Instructor: Martina Seidl
🗓 Time: 19:00 – 19:45
🏫 Room: HS 10   

